In [1]:
import joblib
import json
import warnings

import pandas as pd
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

warnings.filterwarnings("ignore")

/Users/davidtruhlar/Documents/FIIT/BP/5GDigitalTwin/Implementation/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
data = pd.read_csv("../datasets/real_network_data_after_labeling.csv")
scaler = joblib.load("./scaler/scaler.joblib")

In [3]:
# Custom Attention Layer (compatible with LSTM output)
class AttentionLayer(Layer):
    
    """Custom Attention Layer for LSTM models."""

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        
        """Build the layer and define trainable weights for the attention mechanism."""

        self.W = self.add_weight(name='att_weight', shape=(input_shape[-1], 1),
                                 initializer='glorot_uniform', trainable=True)
        self.b = self.add_weight(name='att_bias', shape=(input_shape[1], 1),
                                 initializer='zeros', trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):

        """
        Compute the attention scores and apply them to the input.

        Args:
            x (Tensor): Input tensor of shape (batch_size, time_steps, features).

        Returns:
            Tensor: Context vector of shape (batch_size, features).
        """

        e = K.tanh(K.dot(x, self.W) + self.b)   # Compute attention scores
        a = K.softmax(e, axis=1)                # Normalize attention scores
        output = x * a                          # Apply attention scores
        return K.sum(output, axis=1)            # Sum along the time axis

    def compute_output_shape(self, input_shape):
        
        """
        Compute the output shape of the layer.

        Args:
            input_shape (tuple): Shape of the input tensor.

        Returns:
            tuple: Shape of the output tensor.
        """

        return (input_shape[0], input_shape[-1])

In [4]:
model_base = load_model('trained_models/lstm_base_model.h5')
model_robust = load_model('trained_models/lstm_robust_model.h5')
model_batchnorm = load_model('trained_models/lstm_batchnorm_model.h5')
model_attention = load_model('trained_models/lstm_attention_model.h5', custom_objects={'AttentionLayer': AttentionLayer})

In [5]:
# Recompile the models
model_base.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model_robust.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model_batchnorm.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model_attention.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
with open("./json/uc_map.json", "r") as f:
    UC_MAP = json.load(f)
with open("./json/app_map.json", "r") as f:
    APP_MAP = json.load(f)
with open("./json/log_map.json", "r") as f:
    LOG_MAP = json.load(f)

In [7]:
data['current_uc'] = data['current_uc'].map(UC_MAP)
data["application"] = data["application"].map(APP_MAP)
data["log_type"] = data["log_type"].map(LOG_MAP)

In [8]:
with open('./json/selected_features.json', 'r') as f:
    selected_features = json.load(f)['features']

In [9]:
X = data[selected_features]
y = data['current_uc']
X_scaled = scaler.transform(X)

In [10]:
real_data_class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y), y=y)
real_data_class_weight_dict = dict(zip(np.unique(y), real_data_class_weights))

In [11]:
print("Real data class weights:", real_data_class_weight_dict)

Real data class weights: {np.int64(0): np.float64(0.3595654478007419), np.int64(1): np.float64(1.2806719516798792), np.int64(2): np.float64(0.8759359669506842), np.int64(3): np.float64(1.0105749180816206), np.int64(4): np.float64(13.960905349794238), np.int64(5): np.float64(4.251253132832081)}


In [12]:
def create_sequences(X, y, seq_len):

    """Create sequences of data with given length for LSTM input."""

    X_seq, y_seq = [], []
    for i in range(len(X) - seq_len):
        X_seq.append(X[i:i+seq_len])
        y_seq.append(y[i+seq_len])
    return np.array(X_seq), np.array(y_seq)

X_real_seq, y_real_seq = create_sequences(X_scaled, y, 60)

In [13]:
y_pred_base_model = model_base.predict(X_real_seq)
y_pred_base_model = y_pred_base_model.argmax(axis=1)

y_pred_robust_model = model_robust.predict(X_real_seq)
y_pred_robust_model = y_pred_robust_model.argmax(axis=1)

y_pred_batchnorm_model = model_batchnorm.predict(X_real_seq)
y_pred_batchnorm_model = y_pred_batchnorm_model.argmax(axis=1)

y_pred_attention_model = model_attention.predict(X_real_seq)
y_pred_attention_model = y_pred_attention_model.argmax(axis=1)

211/211 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
211/211 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
211/211 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
211/211 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [14]:
print("Base model classification report:")
print(classification_report(y_real_seq, y_pred_base_model, target_names=list(UC_MAP.keys())))

Base model classification report:
              precision    recall  f1-score   support

         uc1       0.00      0.00      0.00      3115
         uc2       0.14      1.00      0.24       883
         uc3       0.00      0.00      0.00      1291
         uc4       0.00      0.00      0.00      1089
         uc5       0.00      0.00      0.00        81
         uc6       1.00      0.13      0.23       266

    accuracy                           0.14      6725
   macro avg       0.19      0.19      0.08      6725
weighted avg       0.06      0.14      0.04      6725



In [27]:
print("Robust model classification report:")
print(classification_report(y_real_seq, y_pred_robust_model, target_names=list(UC_MAP.keys())))

Robust model classification report:
              precision    recall  f1-score   support

         uc1       0.46      0.94      0.62      3115
         uc2       0.00      0.00      0.00       883
         uc3       0.00      0.00      0.00      1291
         uc4       0.00      0.00      0.00      1089
         uc5       0.00      0.00      0.00        81
         uc6       1.00      0.07      0.13       266

    accuracy                           0.44      6725
   macro avg       0.24      0.17      0.12      6725
weighted avg       0.25      0.44      0.29      6725



In [16]:
print("Batchnorm model classification report:")
print(classification_report(y_real_seq, y_pred_batchnorm_model, target_names=list(UC_MAP.keys())))

Batchnorm model classification report:
              precision    recall  f1-score   support

         uc1       0.31      0.46      0.37      3115
         uc2       0.00      0.00      0.00       883
         uc3       0.08      0.01      0.01      1291
         uc4       0.00      0.00      0.00      1089
         uc5       0.00      0.00      0.00        81
         uc6       0.00      0.00      0.00       266

    accuracy                           0.21      6725
   macro avg       0.06      0.08      0.06      6725
weighted avg       0.16      0.21      0.17      6725



In [17]:
print("Attention model classification report:")
print(classification_report(y_real_seq, y_pred_attention_model, target_names=list(UC_MAP.keys())))

Attention model classification report:
              precision    recall  f1-score   support

         uc1       0.00      0.00      0.00      3115
         uc2       0.00      0.00      0.00       883
         uc3       0.00      0.00      0.00      1291
         uc4       0.16      1.00      0.28      1089
         uc5       0.00      0.00      0.00        81
         uc6       0.00      0.00      0.00       266

    accuracy                           0.16      6725
   macro avg       0.03      0.17      0.05      6725
weighted avg       0.03      0.16      0.05      6725



## Try finetuning with real network data

In [18]:
real_data = pd.read_csv("../datasets/real_network_data_after_labeling.csv")

real_data['application'] = real_data['application'].map(APP_MAP)
real_data['log_type'] = real_data['log_type'].map(LOG_MAP)
real_data['current_uc'] = real_data['current_uc'].map(UC_MAP)
real_data['timestamp'] = pd.to_datetime(real_data['timestamp'])


In [19]:
X_real = real_data[selected_features].values
y_real = real_data['current_uc'].values

In [20]:
# Ensure the selected features match those used during scaler fitting
X_real = real_data[scaler.feature_names_in_]
X_real_scaled = scaler.transform(X_real)

In [21]:
X_real_seq, y_real_seq = create_sequences(X_real_scaled, y_real, 60)

In [22]:
X_ft, X_val, y_ft, y_val = train_test_split(
    X_real_seq, y_real_seq, test_size=0.8, stratify=y_real_seq, random_state=42
)

y_ft_cat = to_categorical(y_ft, len(np.unique(y_ft)))
y_val_cat = to_categorical(y_val, len(np.unique(y_ft)))

In [23]:
model_attention.compile(
    optimizer=Adam(learning_rate=0.01),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [24]:
history = model_attention.fit(
    X_ft,
    y_ft_cat,
    epochs=10,
    batch_size=128,
    validation_data=(X_val, y_val_cat),
    class_weight=real_data_class_weight_dict,
    verbose=1,
    callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
    shuffle=True,
)

Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - accuracy: 0.1324 - loss: 8.6635 - val_accuracy: 0.2310 - val_loss: 1.6259
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.3001 - loss: 1.3430 - val_accuracy: 0.1701 - val_loss: 1.5717
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.2478 - loss: 1.4687 - val_accuracy: 0.0511 - val_loss: 1.6413
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.2046 - loss: 1.4399 - val_accuracy: 0.5022 - val_loss: 1.5501
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.3029 - loss: 1.4353 - val_accuracy: 0.1703 - val_loss: 1.5451
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.2332 - loss: 1.4694 - val_accuracy: 0.5024 - val_loss: 1.5545
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.2205 - loss: 1.4240 - val_accuracy: 0.5022 - val_loss: 1.5340
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.1841 - loss: 1.4106 - val_accuracy: 0.1704 - v

In [25]:
y_real_pred_attention = model_attention.predict(X_real_seq)
y_real_pred_attention = y_real_pred_attention.argmax(axis=1)
print("Attention model classification report on real data:")
print(classification_report(y_real_seq, y_real_pred_attention, target_names=list(UC_MAP.keys())))

211/211 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Attention model classification report on real data:
              precision    recall  f1-score   support

         uc1       0.48      1.00      0.65      3115
         uc2       0.00      0.00      0.00       883
         uc3       0.00      0.00      0.00      1291
         uc4       0.00      0.00      0.00      1089
         uc5       0.00      0.00      0.00        81
         uc6       1.00      0.98      0.99       266

    accuracy                           0.50      6725
   macro avg       0.25      0.33      0.27      6725
weighted avg       0.26      0.50      0.34      6725



<div class="alert alert-info">
<b>Výsledky modelu s Attention:</b> Model pozornosti bol vyhodnotený na reálnych dátach. Dosiahol presnosť 0.50 na testovacej množine, makro priemerná presnosť je 0.27 a vážená priemerná presnosť je 0.34. Výsledky modelu naznačujú, že môže efektívne zachytávať základné vzory v dátach.
</div>

In [26]:
model_robust.compile(
    optimizer=Adam(learning_rate=0.01),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
history = model_robust.fit(
    X_ft,
    y_ft_cat,
    epochs=10,
    batch_size=128,
    validation_data=(X_val, y_val_cat),
    class_weight=real_data_class_weight_dict,
    verbose=1,
    callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
    shuffle=True,
)

y_real_pred_robust = model_robust.predict(X_real_seq)
y_real_pred_robust = y_real_pred_robust.argmax(axis=1)
print("Robust model classification report on real data:")
print(classification_report(y_real_seq, y_real_pred_robust, target_names=list(UC_MAP.keys())))

Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 161ms/step - accuracy: 0.3402 - loss: 5.2518 - val_accuracy: 0.0459 - val_loss: 1.6753
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 149ms/step - accuracy: 0.1301 - loss: 1.4948 - val_accuracy: 0.0480 - val_loss: 1.6057
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 155ms/step - accuracy: 0.1614 - loss: 1.4837 - val_accuracy: 0.1671 - val_loss: 1.6307
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 151ms/step - accuracy: 0.1403 - loss: 1.4967 - val_accuracy: 0.1697 - val_loss: 1.6475
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 151ms/step - accuracy: 0.1572 - loss: 1.5037 - val_accuracy: 0.1704 - val_loss: 1.5815
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 155ms/step - accuracy: 0.1865 - loss: 1.4414 - val_accuracy: 0.2312 - val_loss: 1.5670
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 159ms/step - accuracy: 0.2248 - loss: 1.3363 - val_accuracy: 0.1704 - val_loss: 1.5354
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 159ms/step - accuracy: 0.1783 - loss: 1.4187 - val_accuracy: 0.

<div class="alert alert-info">
<b>Výsledky modelu s Attention:</b> Model pozornosti bol vyhodnotený na reálnych dátach. Dosiahol presnosť 0.17 na testovacej množine, makro priemerná presnosť je 0.19 a vážená priemerná presnosť je 0.07. Výsledky modelu naznačujú, že nemusí efektívne zachytávať základné vzory v dátach.
</div>